In [2]:
from datetime import datetime

In [3]:
from qdrant_client import QdrantClient
client = QdrantClient('http://localhost:6333')
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-rag')])

In [4]:
import requests
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [5]:
print(documents_raw[0].keys())
print(documents_raw[0]['documents'][0].keys())
print([d['course'] for d in documents_raw])

dict_keys(['course', 'documents'])
dict_keys(['text', 'section', 'question'])
['data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp']


In [6]:
from fastembed import TextEmbedding

In [7]:
import pandas as pd
df_supported_models = pd.DataFrame.from_records(TextEmbedding.list_supported_models())
df_supported_models.head()

,model,sources,model_file,description,license,size_in_GB,additional_files,dim,tasks
0,BAAI/bge-base-en,"{'hf': 'Qdrant/fast-bge-base-en', 'url': 'http...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.420,[],768,{}
1,BAAI/bge-base-en-v1.5,"{'hf': 'qdrant/bge-base-en-v1.5-onnx-q', 'url'...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.210,[],768,{}
2,BAAI/bge-large-en-v1.5,"{'hf': 'qdrant/bge-large-en-v1.5-onnx', 'url':...",model.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,1.200,[],1024,{}
3,BAAI/bge-small-en,"{'hf': 'Qdrant/bge-small-en', 'url': 'https://...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,[],384,{}
4,BAAI/bge-small-en-v1.5,"{'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,[],384,{}


In [8]:
EMBEDDING_DIMENSIONALITY = 512
df_supported_models.loc[df_supported_models['dim']==EMBEDDING_DIMENSIONALITY, ['model', 'dim', 'description']]

,model,dim,description
5,BAAI/bge-small-zh-v1.5,512,"Text embeddings, Unimodal (text), Chinese, 512..."
13,Qdrant/clip-ViT-B-32-text,512,"Text embeddings, Multimodal (text&image), Engl..."
16,jinaai/jina-embeddings-v2-small-en,512,"Text embeddings, Unimodal (text), English, 819..."


In [9]:
model_handle = 'jinaai/jina-embeddings-v2-small-en'
print(model_handle)

jinaai/jina-embeddings-v2-small-en


In [13]:
from qdrant_client import models
collection_name = 'zoomcamp-rag'
client.delete_collection(collection_name=collection_name)
client.create_collection(
	collection_name=collection_name,
	vectors_config=models.VectorParams(
		size=EMBEDDING_DIMENSIONALITY,
		distance=models.Distance.COSINE
	)
)

True

In [14]:
points = []
id = 0
number_of_docs_used_per_course = 50 # just to limit use of codespace

for course in documents_raw:
	for doc in course['documents'][:number_of_docs_used_per_course]:
		point = models.PointStruct(
            id=id,
			vector=models.Document(text=doc['text'], model=model_handle),
			payload={
				'text' : doc['text'],
				'section' : doc['section'],
				'course' : course['course'],
			}
		)
		points.append(point)
		id = id+1

In [15]:
print(datetime.now())
client.upsert(
	collection_name=collection_name,
	points=points
)
print(datetime.now())

2025-06-27 01:29:49.934865
2025-06-27 01:30:11.073586


In [16]:
def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )
    return results

In [17]:
import random
import json

random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [18]:
result = search(course_piece['question'])
result

QueryResponse(points=[ScoredPoint(id=143, version=0, score=0.79207486, payload={'text': 'I found a good guide how to get acces to your machine again when you removed your public key.\nUsing the following link you can go to Session Manager and log in to your instance and create public key again. https://repost.aws/knowledge-center/ec2-linux-fix-permission-denied-errors\nThe main problem for me here was to get my old public key, so for doing this you should run the following command: ssh-keygen -y -f /path_to_key_pair/my-key-pair.pem\nFor more information: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/describe-keys.html#retrieving-the-public-key\nHanna Zhukavets (a.zhukovec1901@gmail.com)', 'section': 'Module 1: Introduction', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [19]:
[doc.payload for doc in result.points]

[{'text': 'I found a good guide how to get acces to your machine again when you removed your public key.\nUsing the following link you can go to Session Manager and log in to your instance and create public key again. https://repost.aws/knowledge-center/ec2-linux-fix-permission-denied-errors\nThe main problem for me here was to get my old public key, so for doing this you should run the following command: ssh-keygen -y -f /path_to_key_pair/my-key-pair.pem\nFor more information: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/describe-keys.html#retrieving-the-public-key\nHanna Zhukavets (a.zhukovec1901@gmail.com)',
  'section': 'Module 1: Introduction',
  'course': 'mlops-zoomcamp'}]

In [20]:
print(f"=== Question ===\n{course_piece['question']}\n")
print(f"=== Top Retrieved Answer ===\n{result.points[0].payload['text']}\n")
print(f"=== Original answer ===\n{course_piece['text']}")

=== Question ===
Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records."

=== Top Retrieved Answer ===
I found a good guide how to get acces to your machine again when you removed your public key.
Using the following link you can go to Session Manager and log in to your instance and create public key again. https://repost.aws/knowledge-center/ec2-linux-fix-permission-denied-errors
The main problem for me here was to get my old public key, so for doing this you should run the following command: ssh-keygen -y -f /path_to_key_pair/my-key-pair.pem
For more information: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/describe-keys.html#retrieving-the-public-key
Hanna Zhukavets (a.zhukovec1901@gmail.com)

=== Original answer ===
Even though the upload works using aws cli and boto3 in Jupyter notebook.
Solution set the AWS_PROFILE environment variable (the default profile is 

In [21]:
print(search('what if i submit homework late').points[0].payload['text'])

No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [22]:
client.create_payload_index(
    collection_name=collection_name,
    field_name='course',
    field_schema='keyword'
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
def search_in_course(query, course='mlops-zoomcamp', limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key='course',
                    match=models.MatchValue(value=course)
                )
            ]
        )
    )
    return results

In [24]:
print(search_in_course('what if i submit homework late', course='mlops-zoomcamp', limit=1).points[0].payload['text'])

Please choose the closest one to your answer. Also do not post your answer in the course slack channel.
